In [1]:
!pip list

Package             Version   Location                             
------------------- --------- -------------------------------------
argon2-cffi         20.1.0    
async-generator     1.10      
attrs               20.3.0    
backcall            0.2.0     
bleach              3.3.0     
certifi             2020.12.5 
cffi                1.14.5    
chardet             4.0.0     
cycler              0.10.0    
decorator           4.4.2     
defusedxml          0.7.1     
entrypoints         0.3       
idna                2.10      
ipykernel           5.5.0     
ipython             7.21.0    
ipython-genutils    0.2.0     
ipywidgets          7.6.3     
jedi                0.18.0    
Jinja2              2.11.3    
joblib              1.0.1     
jsonschema          3.2.0     
jupyter             1.0.0     
jupyter-client      6.1.12    
jupyter-console     6.3.0     
jupyter-core        4.7.1     
jupyterlab-pygments 0.1.2     
jupyterlab-widgets  1.0.0     
kiwisolver          1.3.1  

In [3]:
import sys
print(sys.version, sys.platform, sys.executable)

3.8.2 (default, Apr 27 2020, 15:53:34) 
[GCC 9.3.0] linux /mnt/d/spikeinterface/hussaini-to-nwb/venv/bin/python3


In [4]:
'''
RecordingExtractor objects
==========================

The :code:`RecordingExtractor` is the basic class for handling recorded data. Here is how it works.
'''

import numpy as np
import spikeinterface.extractors as se

##############################################################################
# We will create a :code:`RecordingExtractor` object from scratch using :code:`numpy` and the
# :code:`NumpyRecordingExtractor`
#
# Let's define the properties of the dataset

num_channels = 7
sampling_frequency = 30000  # in Hz
duration = 20
num_timepoints = int(sampling_frequency * duration)

##############################################################################
# We can generate a pure-noise timeseries dataset recorded by a linear probe geometry

timeseries = np.random.normal(0, 10, (num_channels, num_timepoints))
geom = np.zeros((num_channels, 2))
geom[:, 0] = range(num_channels)

##############################################################################
# And instantiate a :code:`NumpyRecordingExtractor`:

recording = se.NumpyRecordingExtractor(timeseries=timeseries, geom=geom, sampling_frequency=sampling_frequency)

##############################################################################
# We can now print properties that the :code:`RecordingExtractor` retrieves from the underlying recording.

print('Num. channels = {}'.format(len(recording.get_channel_ids())))
print('Sampling frequency = {} Hz'.format(recording.get_sampling_frequency()))
print('Num. timepoints = {}'.format(recording.get_num_frames()))
print('Stdev. on third channel = {}'.format(np.std(recording.get_traces(channel_ids=2))))
print('Location of third electrode = {}'.format(recording.get_channel_property(channel_id=2, property_name='location')))

##############################################################################
# Some extractors also implement a :code:`write` function. We can for example save our newly created recording into
# MDA format (Mountainsort4 format):

se.MdaRecordingExtractor.write_recording(recording=recording, save_path='sample_mountainsort_dataset')

##############################################################################
# and read it back with the proper extractor:

recording2 = se.MdaRecordingExtractor(folder_path='sample_mountainsort_dataset')
print('Num. channels = {}'.format(len(recording2.get_channel_ids())))
print('Sampling frequency = {} Hz'.format(recording2.get_sampling_frequency()))
print('Num. timepoints = {}'.format(recording2.get_num_frames()))
print('Stdev. on third channel = {}'.format(np.std(recording2.get_traces(channel_ids=2))))
print('Location of third electrode = {}'.format(recording.get_channel_property(channel_id=2, property_name='location')))

##############################################################################
# Sometimes experiments are run with different conditions, e.g. a drug is applied, or stimulation is performed. 
# In order to define different phases of an experiment, one can use epochs:

recording2.add_epoch(epoch_name='stimulation', start_frame=1000, end_frame=6000)
recording2.add_epoch(epoch_name='post_stimulation', start_frame=6000, end_frame=10000)
recording2.add_epoch(epoch_name='pre_stimulation', start_frame=0, end_frame=1000)

recording2.get_epoch_names()

##############################################################################
# An Epoch can be retrieved and it is returned as a :code:`SubRecordingExtractor`, which is a subclass of the
# :code:`RecordingExtractor`, hence maintaining the same functionality.

recording3 = recording2.get_epoch(epoch_name='stimulation')
epoch_info = recording2.get_epoch_info('stimulation')
start_frame = epoch_info['start_frame']
end_frame = epoch_info['end_frame']

print('Epoch Name = stimulation')
print('Start Frame = {}'.format(start_frame))
print('End Frame = {}'.format(end_frame))
print('Mean. on second channel during stimulation = {}'.format(np.mean(recording3.get_traces(channel_ids=1))))
print('Location of third electrode = {}'.format(recording.get_channel_property(channel_id=2, property_name='location')))

##############################################################################
# :code:`SubRecordingExtractor` objects can be used to extract arbitrary subsets of your data/channels manually without
# epoch functionality:

recording4 = se.SubRecordingExtractor(parent_recording=recording2, channel_ids=[2, 3, 4, 5], start_frame=14000,
                                      end_frame=16000)

print('Num. channels = {}'.format(len(recording4.get_channel_ids())))
print('Sampling frequency = {} Hz'.format(recording4.get_sampling_frequency()))
print('Num. timepoints = {}'.format(recording4.get_num_frames()))
print('Stdev. on third channel = {}'.format(np.std(recording4.get_traces(channel_ids=2))))
print(
    'Location of third electrode = {}'.format(recording4.get_channel_property(channel_id=2, property_name='location')))

##############################################################################
# or to remap the channel ids:

recording5 = se.SubRecordingExtractor(parent_recording=recording2, channel_ids=[2, 3, 4, 5],
                                      renamed_channel_ids=[0, 1, 2, 3],
                                      start_frame=14000, end_frame=16000)
print('New ids = {}'.format(recording5.get_channel_ids()))
print('Original ids = {}'.format(recording5.get_original_channel_ids([0, 1, 2, 3])))
print('Num. channels = {}'.format(len(recording5.get_channel_ids())))
print('Sampling frequency = {} Hz'.format(recording5.get_sampling_frequency()))
print('Num. timepoints = {}'.format(recording5.get_num_frames()))
print('Stdev. on third channel = {}'.format(np.std(recording5.get_traces(channel_ids=0))))
print(
    'Location of third electrode = {}'.format(recording5.get_channel_property(channel_id=0, property_name='location')))


Num. channels = 7
Sampling frequency = 30000.0 Hz
Num. timepoints = 600000
Stdev. on third channel = 9.998943877297208
Location of third electrode = [2. 0.]
Num. channels = 7
Sampling frequency = 30000.0 Hz
Num. timepoints = 600000
Stdev. on third channel = 9.998945236206055
Location of third electrode = [2. 0.]
Epoch Name = stimulation
Start Frame = 1000
End Frame = 6000
Mean. on second channel during stimulation = 0.00782747846096754
Location of third electrode = [2. 0.]
Num. channels = 4
Sampling frequency = 30000.0 Hz
Num. timepoints = 2000
Stdev. on third channel = 9.996794700622559
Location of third electrode = [2. 0.]
New ids = [0, 1, 2, 3]
Original ids = [2, 3, 4, 5]
Num. channels = 4
Sampling frequency = 30000.0 Hz
Num. timepoints = 2000
Stdev. on third channel = 9.996794700622559
Location of third electrode = [2. 0.]


In [9]:
from spikeextractors.extractors.neoextractors.neobaseextractor import NeoBaseRecordingExtractor, NeoBaseSortingExtractor

try:
    import neo
    HAVE_NEO = True
except ImportError:
    HAVE_NEO = False

class AxonaRecordingExtractor(NeoBaseRecordingExtractor):
    """
    The axona extractor is wrapped from neo.rawio.AxonaRawIO.
    
    Parameters
    ----------
    filename: str
        the "base" filename of .bin and .set files (e.g. my_data for my_data.bin and
        my_data.set).    
    """    
    extractor_name = 'PlexonRecording'
    mode = 'file'
    installed = HAVE_NEO
    NeoRawIOClass = 'PlexonRawIO'

In [13]:
# Establish filename

import os

dir_name = r'../../freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw'
base_filename = 'axona_raw'
filename = os.path.join(dir_name, base_filename)

In [15]:
# Create axona recording extractor object

axore = AxonaRecordingExtractor(filename=filename)

FileNotFoundError: [Errno 2] No such file or directory: '../../freelance-work/catalyst-neuro/hussaini-lab-to-nwb/example_data_raw/axona_raw'